In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [ ]:
print(X)

In [ ]:
print(y)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')      # One hot encoding shifts the variable converted to the first column
X = np.array(ct.fit_transform(X))                                                                      # Hence, the 1.0 0.0 0.0 come before credit score column
print(X)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)                       # ALWAYS apply feature scaling to all columns in ANN's irrespective of values of the column

In [12]:
ann = tf.keras.models.Sequential()

In [13]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [14]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [15]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) # If we had a non binary output var, we would need that many units and we would one hot encode them